In [1]:
# ============================================
# 1) 설치
# ============================================
!pip -q install langchain langchain-community langchain-text-splitters faiss-cpu sentence-transformers pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00


In [ ]:

# ============================================
# 2) PDF 다운로드 (요청하신 동일 파일 & 재시도 로직)
# ============================================
import requests, os, time
import urllib.request

urllib.request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch07/2020_%EA%B2%BD%EC%A0%9C%EA%B8%88%EC%9C%B5%EC%9A%A9%EC%96%B4%20700%EC%84%A0_%EA%B2%8C%EC%8B%9C.pdf", filename="2020_경제금융용어 700선_게시.pdf")


## Parent Document Retriever 접근방법 요약

  🎯 핵심 문제

  - 작은 청크: 검색은 정확하지만 컨텍스트 부족
  - 큰 청크: 컨텍스트는 풍부하지만 검색 정확도 저하

  🔄 해결 전략

  이중 분할 구조로 검색 정확도와 컨텍스트 풍부함을 동시 확보

  핵심 아키텍처

  원본 문서
      ↓
  Parent 청크 (1200자) ← 실제 반환되는 문서
      ↓
  Child 청크 (400자)   ← 검색에 사용되는 문서

  질문 → vectorstore(자식 검색) → 관련 부모 doc_id 찾기 → docstore에서 부모 청크 꺼내 반환

  🛠 구현 메커니즘

  1. 이중 분할: Parent(1200자) + Child(400자) 청크 생성
  2. 이중 저장소:
    - VectorStore: Child 청크 임베딩 저장 (검색용)
    - InMemoryStore: Parent 청크 원본 저장 (반환용)
  3. 검색 흐름: Child로 검색 → 매칭된 Parent 반환

  💡 핵심 장점

  - 정확한 검색: 작은 Child 청크로 정밀 검색
  - 풍부한 컨텍스트: 큰 Parent 청크로 충분한 정보 제공
  - 자동 매핑: Child-Parent 관계 자동 관리

In [9]:
import os

pdf_path = "2020_경제금융용어 700선_게시.pdf"

if not os.path.exists(pdf_path) or os.path.getsize(pdf_path) == 0:
    r = requests.get(url, stream=True)
    r.raise_for_status()
    with open(pdf_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
print(" PDF ready:", pdf_path)

# ============================================
# 3) PDF 로드 & Parent/Child 청크 분할
# ============================================
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader(pdf_path)
docs = loader.load()
print(f" 전체 페이지 수: {len(docs)}")

# Parent = 긴 덩어리, Child = 검색용 작은 덩어리
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=120)
child_splitter  = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=40)

# ============================================
# 4) 임베딩 & 빈 FAISS 인덱스 생성
# ============================================
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import OpenAIEmbeddings

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# embedding = OpenAIEmbeddings(model="text-embedding-3-small", chunk_size=100)

# 정석: IndexFlatL2로 빈 인덱스 생성
from langchain_community.vectorstores.faiss import FAISS, dependable_faiss_import
from langchain.docstore import InMemoryDocstore

faiss = dependable_faiss_import()
dim = len(embedding.embed_query("dummy"))  # 임베딩 차원
index = faiss.IndexFlatL2(dim)
vectorstore = FAISS(embedding.embed_query, index, InMemoryDocstore(), {})

# ============================================
# 5) InMemoryStore (부모 문서 저장소)
# ============================================
try:
    from langchain.storage import InMemoryStore
except ImportError:
    class InMemoryStore(dict):
        def mset(self, items):
            for k, v in items:
                self[k] = v
docstore = InMemoryStore()

# ============================================
# 6) ParentDocumentRetriever 생성
# ============================================
try:
    from langchain.retrievers import ParentDocumentRetriever
except ImportError:
    from langchain_community.retrievers import ParentDocumentRetriever

parent_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=docstore,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

# ============================================
# 7) 문서 색인
# ============================================
print("부모-자식 청크 색인 중...")
parent_retriever.add_documents(docs)
print(" 색인 완료")

# ============================================
# 8) 검색 테스트
# ============================================
def ask(q, k=3):
    hits = parent_retriever.get_relevant_documents(q)
    print(f"\n Q: {q}\n--- Top-{min(k,len(hits))} 부모 문서 ---")
    for i, d in enumerate(hits[:k], 1):
        page = d.metadata.get("page", "?")
        context = d.page_content[:160].replace('\n',' ')
        print(f"{i:>2}. p.{page} | {context}...")

ask("인플레이션의 정의와 원인, 그리고 기준금리와의 관계는 무엇인가요?")
# ask("환율 변동이 물가에 어떤 영향을 미치나요?")
# ask("스태그플레이션이 무엇이며 정책 대응의 어려움은 무엇인가요?")


 PDF ready: 2020_경제금융용어 700선_게시.pdf
 전체 페이지 수: 371


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


부모-자식 청크 색인 중...
 색인 완료

 Q: 인플레이션의 정의와 원인, 그리고 기준금리와의 관계는 무엇인가요?
--- Top-3 부모 문서 ---
 1. p.212 | 196 경제금융용어 700선 원리금 연체 등 객관적인 손상(impairment)의 증거가 있는 경우에만 대손충당금 적립을  허용하고 있어 대손충당금에 예상손실을 반영하는 데 어려움이 있었다. 이에 따라 바젤  자본규제는 대손충당금이 예상손실에 미달(shortfall)시 동 금액을 기본자...
 2. p.344 | 328 경제금융용어 700선 현재 우리나라 은행권에는 오만원권에 띠형 홀로그램이, 만원과 오천원권에는 패치형  홀로그램이 각각 적용되어 있다. 오만원권의 띠형 홀로그램은 앞면 왼쪽 끝 부분에 부착되 어 있으며 보는 각도에 따라 상･중･하 3곳에서 ① 우리나라 지도 ② 태극 ③ 4괘 무늬...
 3. p.336 | 320 경제금융용어 700선 뜻한다. 이는 혁신의 정도에 따라 전통적(traditional) 핀테크와 신흥(emergent) 핀테크로  구분할 수 있다. 전통적 핀테크는 기존 금융서비스의 가치사슬 안에서 그 서비스의  효율을 높이는 역할을 한다. 즉 기존 금융서비스를 자동화하려는 금융회...


In [5]:

ask("가계신용통계 에 대해 설명해")


 Q: 가계신용통계 에 대해 설명해
--- Top-3 부모 문서 ---
 1. p.213 | 197 ㅇ  옵션매도자에게 프리미엄을 지급하며 반대로 옵션매도자는 프리미엄을 받는 대신 옵션 매입자의 옵션 행사에 따라 발생하는 자신의 의무를 이행할 책임을 부담한다. 옵션거래 의 손익은 행사가격, 현재가격 및 프리미엄에 의해 결정된다. 한편 옵션의 가격이 어떻게  결정되는가에 대해서는...
 2. p.271 | 255 ㅈ  제1차 통화조치  한국전쟁의 여파로 산업활동이 크게 위축되고 물가가 급등하는 등 경제가 큰 혼란에  빠짐에 따라 이를 타개하기 위하여 1953년 2월 15일 화폐단위를 ‘원(圓)’에서 ‘환(圜)’으로  변경하고 100대 1로 절하(100圓 → 1圜)하는 긴급통화조치를 단행하...
 3. p.116 | 100 경제금융용어 700선  연관검색어 : 조세부담률 레그테크 레그테크(RT; RegTech, Regulatory Technology)는 금융업 등 산업 전반에 걸쳐 혁신  정보기술(IT)과 규제를 결합하여 규제관련 요구사항 및 절차를 향상시키는 기술 또는  회사를 뜻한다. 이는 금융...


In [3]:
from langchain_core.tracers.stdout import ConsoleCallbackHandler
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

template = """당신은 한국은행에서 만든 금융 용어를 설명해주는 금융해설자입니다.
주어진 검색 결과를 바탕으로 답변하세요.
검색 결과에 없는 내용이라면 답변할 수 없다고 하세요. 반말로 친근하게 답변하세요.
{context}

Question: {question}
Answer:
"""

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

prompt = PromptTemplate.from_template(template)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type_kwargs={"prompt": prompt},
    retriever=parent_retriever,
    return_source_documents=True)

In [ ]:
import gradio as gr

# 인터페이스 생성
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="경제금융용어 챗봇") # 챗봇 레이블을 좌측 상단에 구성
    msg = gr.Textbox(label="질문해주세요!")  # 하단의 채팅창 레이블
    clear = gr.Button("대화 초기화")  # 대화 초기화 버튼

    # 챗봇의 답변을 처리하는 함수
    def respond(message, chat_history):
      result = qa_chain(message, 
                        callbacks=[ConsoleCallbackHandler()])
      bot_message = result['result']

      # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가
      chat_history.append((message, bot_message))
      return "", chat_history

    # 사용자의 입력을 제출(submit)하면 respond 함수가 호출
    msg.submit(respond, [msg, chatbot], [msg, chatbot])

    # '초기화' 버튼을 클릭하면 채팅 기록을 초기화
    clear.click(lambda: None, None, chatbot, queue=False)

# 인터페이스 실행
demo.launch(debug=True)